In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json

import re

import asyncio
from playwright.async_api import async_playwright
import tqdm.asyncio

# Subset to software

In [103]:
df = pd.read_csv("./websites.csv")

In [105]:
len(df)

66342

In [104]:
df.head(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,1543,Add*Ed,http://www.addedrg.ca,['autism' 'intellectual disability' 'teacher t...,"{'city': 'Kingston', 'country': 'canada', 'cou...",Education Management,NaN
1,1952,"Addison AG Tech, LP",http://www.AddisonFund.com,[],"{'city': 'Addison', 'country': 'united states'...",Financial Services,NaN


In [4]:
df.tail(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
66340,7404336,Wireframe Marketing,https://wireframemarketing.com,['SEO Services' 'Logo Design' 'Branding' 'Soci...,"{'city': 'REXBURG', 'country': 'united states'...",Marketing & Advertising,NaN
66341,7404394,Wirehound Networks,http://www.wirehoundnetworks.com,[],"{'city': 'Richfield', 'country': 'united state...",Computer & Network Security,NaN


In [8]:
df = df[df['Industry'] == 'Computer Software'].reset_index()
len(df)

6548

In [9]:
df.head(2)

,index,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,11,5314,Sawa Impact Labs,https://sawa.gallery/,[],"{'city': 'Stanford', 'country': 'united states...",Computer Software,NaN
1,19,5505,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,[],"{'city': 'Ann Arbor', 'country': 'united state...",Computer Software,NaN


In [42]:
def parse_single_quote_json(json_str):
    json_str_fixed = json_str.replace("'", '"').replace("None", "null")
    try:
        return json.loads(json_str_fixed)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
def _get_country(_str):
    x = parse_single_quote_json(_str)
    if x is None:
        return None
    if 'country' not in x.keys():
        return None
    return x['country']
    

In [43]:
df['country'] = df.Address.apply(lambda x: _get_country(x))

Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Invalid control character at: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 13 (char 12)
Error parsing JSON: Invalid control character at: line 1 column 12 (char 11)
Error parsing JSON: Invalid control character at: line 1 column 81 (char 80)


country is good (was checking if it's full of Chinese/Indian companies)

In [44]:
df.country.value_counts()

country
united states     4734
united kingdom     884
canada             392
australia          254
singapore          172
ireland             79
new zealand         26
Name: count, dtype: int64

In [47]:
df.Website.head(20)

0                                https://sawa.gallery/
1     https://maizepages.umich.edu/organization/openmi
2                                   https://siftmo.com
3                              https://www.aleios.com/
4                                    http://niftys.com
5                                 https://basewell.com
6                                    http://journey.io
7                             https://apple.co/3HcKeLx
8                               https://sqdc.avoyer.ca
9                             http://www.appcoderz.com
10                                  https://appdive.io
11                              https://greenpixie.com
12                                http://tryparade.com
13                       http://www.extendingfamily.us
14                                 https://nixmade.com
15                             https://www.jumproom.io
16                                    https://maple.so
17                         http://www.blueprintgtm.com
18        

In [102]:
df = df[df['country'] == 'united states']
len(df)

4734

# Scrape website

In [59]:
async def scrape_url(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            context = browser.new_context()
            page = await browser.new_page()
            
            await page.goto(url)
            
            await page.wait_for_load_state(timeout=20000)
            await page.keyboard.press('PageDown')
            await page.wait_for_load_state(timeout=20000)
            
            href = await page.evaluate('() => document.location.href')
            await page.wait_for_load_state(timeout=20000)
            await page.screenshot(path="screenshot.png", full_page=True)
            
            content = await page.content()
            await browser.close()
            return content
    except Exception as e:
        print(e)
        return None

In [60]:
url = 'https://maizepages.umich.edu/organization/openmi'

In [61]:
content = await scrape_url(url)

/home/krisztian/miniconda3/envs/all/lib/python3.12/inspect.py:479: RuntimeWarning: coroutine 'Browser.new_context' was never awaited
  def isframe(object):


In [84]:
async def limit_parallel(tasks, n=5):
    """
    Run up to n async tasks in parallel.

    :param n: Maximum number of concurrent tasks.
    :param tasks: List of coroutine functions to execute.
    :return: List of results from the tasks.
    """
    semaphore = asyncio.Semaphore(n)

    async def sem_task(task):
        async with semaphore:
            return await task

    # Wrap tasks with semaphore logic
    wrapped_tasks = [sem_task(task) for task in tasks]

    # Run all tasks concurrently
    return await tqdm.asyncio.tqdm.gather(*wrapped_tasks)

In [85]:
urls = df.Website.head(20)

In [95]:
tasks = [scrape_url(u) for u in urls]

In [96]:
results = await limit_parallel(tasks, 10)

  5%|███████████▌                                                                                                                                                                                                                           | 1/20 [00:02<00:45,  2.40s/it]

Page.goto: net::ERR_NAME_NOT_RESOLVED at https://nixmade.com/
Call log:
navigating to "https://nixmade.com/", waiting until "load"



 10%|███████████████████████                                                                                                                                                                                                                | 2/20 [00:02<00:21,  1.20s/it]

Page.goto: net::ERR_CERT_COMMON_NAME_INVALID at http://www.appcoderz.com/
Call log:
navigating to "http://www.appcoderz.com/", waiting until "load"



 45%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                               | 9/20 [00:07<00:09,  1.17it/s]

Page.goto: net::ERR_NAME_NOT_RESOLVED at http://www.extendingfamily.us/
Call log:
navigating to "http://www.extendingfamily.us/", waiting until "load"



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.44it/s]


In [97]:
results[0]

'<!DOCTYPE html><html class="yui3-js-enabled js-focus-visible js flexbox canvas canvastext webgl no-touch hashchange history draganddrop rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms no-csstransforms3d csstransitions video audio svg inlinesvg svgclippaths" data-js-focus-visible="" style=""><div id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important"></div><head>\n\n  <title>Squarespace - Website Expired</title>\n  <meta http-equiv="X-UA-Compatible" content="chrome=1">\n\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/legacy.js" nomodule="" type="text/javascript" crossorigin="anonymous"></script>\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/modern.js" type="text/javascript" crossorigin="anonymous"></script>\n  <script crossorigin="anonymous" src="//assets.squarespace.com/universal/scripts-compressed/extract-css-runt

In [98]:
def _estime_whole_df(secs):
    return secs / 20 * len(df) / 3600

In [100]:
_estime_whole_df(13)

1.1822777777777778

Going to
1. Set of async job for whole list
2. do md conversion
3. do prompt + evals for extracting jobs links / classifying pages / etc

page classifications:
- is it a valid htmls
- does it contains jobs?
    - Yes / list
    - Yes / open apply
    - No / with links
    - No / no links
- extract links

# Do markdown conversion

In [67]:
df = pd.read_csv("../data/01_subset_enriched.csv")

In [68]:
df.head(2)

,CompanyName,Website,scrape_status,homepage_content
0,Sawa Impact Labs,https://sawa.gallery/,Not Started,NaN
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Not Started,NaN


In [69]:
df.scrape_status.value_counts()

scrape_status
Not Started    4080
Success         499
Failed          155
Name: count, dtype: int64

In [70]:
failed = df[df.scrape_status == 'Failed']

In [71]:
len(failed)

155

In [72]:
success = df[df.scrape_status== 'Success'].reset_index(drop=True)

In [73]:
len(success)

499

In [74]:
success.head()

,CompanyName,Website,scrape_status,homepage_content
0,"Shara, Inc (weshara.com)",https://www.weshara.com,Success,"<!DOCTYPE html><html lang=""en"" style=""--inner1..."
1,Good Kiwi LLC,https://www.goodkiwi.llc,Success,"<!DOCTYPE html><html lang=""en-us""><head><meta ..."
2,Activepieces,https://www.activepieces.com,Success,"<!DOCTYPE html><html lang=""en"" data-capo=""""><h..."
3,The Brilliant Company,https://thebrilliant.company,Success,"<!DOCTYPE html><html lang=""en""><head>\n ..."
4,Aclarify,https://aclarify.com/,Success,"<!DOCTYPE html><html lang=""en"" class=""h-full b..."


In [80]:
from jobsfinder.core import html2md

In [82]:
print(html2md(success.homepage_content.iloc[0]))


Welcome to SHARA![](images/50167bb29d9df130210406561b502988.png)[![](images/1e28468f394d7bc0fe364a8daaca7fdb.png)](https://weshara.com)[Listings](https://weshara.com/listings)  


[Sellers](https://weshara.com/sellers)  


[Agents](https://weshara.com/agents)  


[Community](https://community.weshara.com)  


[SIGN IN](https://portal.weshara.com)  


We are changing the way people buy, sell, and use a second home.

[get in touch](https://share.hsforms.com/1EIMJC7CZSOqz-gD-7pSQHQdguvb)  


SHARA is the co\-ownership platform that makes it possible for people to invest and use properties without all of the headaches.  


With SHARA, ownership becomes not only possible but practical. 

  


Enjoy more "bang for your buck" in purchase power. Arrive, at your vacation, on vacation, ready to relax because the work is all done. Less hassle, more fun, and when you're ready to spend your precious time somewhere else, simply sell your share.   


  


Plan long stays or be spontaneous and pop in

In [105]:
df = success.copy()

In [85]:
from tqdm import tqdm

In [ ]:
df['md'] = [html2md(x) for x in tqdm(df.homepage_content)]

 15%|███████████████████████████████████▎                                                                                                                                                                                                 | 77/499 [00:28<03:40,  1.91it/s]

In [ ]:
df['html_length'] = df.homepage_content.apply(lambda x: None if x is None else len(x))
df['md_length'] = df.md.apply(lambda x: None if x is None else len(x))

In [ ]:
df.html_length.describe()

In [ ]:
df.md_length.describe()

In [108]:
df.md_length.describe()

count       499.000000
mean       5797.186373
std        8562.724653
min           2.000000
25%        1431.000000
50%        3891.000000
75%        6967.500000
max      104058.000000
Name: md_length, dtype: float64

In [109]:
df['md2html_ratio'] = df.html_length / df.md_length

In [110]:
df.md2html_ratio.describe()

count     499.000000
mean       79.635834
std       285.404051
min         1.450450
25%         9.180246
50%        18.979986
75%        52.218990
max      3619.895900
Name: md2html_ratio, dtype: float64

In [111]:
sub1 = df[df.md_length > 20000]

In [112]:
len(sub1)

17

In [116]:


import re

def replace_empty_newlines(text):
    return re.sub(r'(\n{4,})', '\n\n\n', text)


In [117]:
print(replace_empty_newlines(sub1.md.iloc[3]))


Rutter \| Unified API for B2B Financial Products


Bank Feeds \& Accounting Integrations: Why neobanks need both[Learn More](https://www.rutter.com/events/bank-feeds-accounting-integrations-why-neobanks-need-both?utm_source=nav_banner)

Bank Feeds \& Accounting Integrations: Why neobanks need both[Sign Up](https://www.rutter.com/events/bank-feeds-accounting-integrations-why-neobanks-need-both?utm_source=nav_banner)

Back[![Rutter logo black]()](/)Product

Our integrations[Integration LibraryView all supported platforms you can integrate with](/integrations)[Payments APIRead transactions, subscriptions, balance, and payouts](/product/payments-api)[Accounting APIRead financial statements, bills, invoices](/product/accounting-api)[Ads APIUnderstand campaign and ad account performance](/product/ads-api)[Commerce APIUnderstand orders, inventory, and customer data](/product/commerce-api)[Bank Feeds APISync bank transactions with your customers' accounting platforms](/qbo-bank-feeds)Our feat